# Stratospheric Balloon Simulation Demo

This notebook demonstrates the balloon_sim package for simulating stratospheric balloon trajectories driven by wind data.

In [ ]:
# autoreload 2
%reload_ext autoreload
%autoreload 2


In [ ]:
# Import the balloon simulation package
from balloon_sim import (
    WindField,
    Balloon,
    Fleet,
    CoverageAnalyzer,
    DEFAULT_COVERAGE_RADIUS_KM,
    download_ncep_data,
)
from balloon_sim.visualization import (
    plot_trajectories,
    plot_coverage,
    plot_coverage_timeseries,
    create_trajectory_animation_parallel,
    create_trajectory_animation,
    create_time_since_visit_animation,
)

import matplotlib.pyplot as plt
import numpy as np

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## 1. Load Wind Data

Download and load NCEP Reanalysis 2 wind data. The `download_ncep_data` function will automatically download the data if it doesn't exist locally. The wind field supports optional linear interpolation between time steps for smoother trajectories.

In [ ]:
# Download NCEP wind data if not already present
# This downloads ~400MB of data (2 years of global wind fields)
data_path = download_ncep_data('../data/', years=[2024])

# Load wind data with linear interpolation for smoother trajectories
wind = WindField(data_path, interpolation='linear')

print(f"Wind grid: {wind.grid_height} x {wind.grid_width}")
print(f"Time steps: {wind.num_times}")
print(f"Pressure level: {wind.pressure_level} hPa")

## 2. Single Balloon Simulation

Simulate a single balloon starting from a specific location. All coordinates use standard format:
- Latitude: -90 to 90 (negative = south)
- Longitude: -180 to 180 (negative = west)

In [ ]:
# Create and simulate a single balloon
# Starting near Denver, Colorado
balloon = Balloon(lat=39.7, lon=-104.9, balloon_id='Denver')
balloon.simulate(wind, num_steps=720)  # 720 hours = 30 days

# Export to DataFrame
df = balloon.to_dataframe()
print(f"Trajectory has {len(df)} points")
print(f"\nFirst few points:")
print(df.head())

print(f"\nLast few points:")
print(df.tail())

In [ ]:
# Plot the single balloon trajectory
fig = plot_trajectories(balloon, projection='robinson', title='Single Balloon Trajectory (30 days)')
plt.show()

## 3. Fleet Simulation

Simulate a fleet of balloons distributed across a region.

In [ ]:
# Create a fleet launched from california
fleet = Fleet.create_random(
    n_balloons=15,
    lat_range=(32, 48),
    lon_range=(-124.5, -114),
    seed=42,
)

# Create a fleet launched from Argentina
sa_fleet = Fleet.create_random(
    n_balloons=15,
    lat_range=(-55, -15),
    lon_range=(-73.5, -53),
    seed=123,
)

# Merge fleets
fleet.balloons.extend(sa_fleet.balloons)


print(f"Created fleet with {len(fleet)} balloons")

# Simulate 
num_days = 60
fleet.simulate(wind, num_steps=24 * num_days)

# Export all trajectories
fleet_df = fleet.to_dataframe()
print(f"\nTotal trajectory points: {len(fleet_df)}")
print(fleet_df.head())

In [ ]:
# Plot fleet trajectories
fig = plot_trajectories(
    fleet, 
    projection='robinson', 
    title=f'Fleet of {len(fleet)} Balloons ({num_days} days)',
    marker_size=0.5
)
plt.show()

## 4. Coverage Analysis

Analyze the geographic coverage provided by the balloon fleet.

In [ ]:
# Create coverage analyzer with default coverage radius (370 km)
analyzer = CoverageAnalyzer(coverage_radius_km=DEFAULT_COVERAGE_RADIUS_KM)

# Compute cumulative coverage
coverage_grid = fleet.compute_coverage(analyzer)

# Get coverage statistics
stats = analyzer.compute_coverage_statistics(coverage_grid)
print("Coverage Statistics:")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Plot coverage map
fig = plot_coverage(
    coverage_grid,
    analyzer=analyzer,
    projection='robinson',
    cmap='viridis'
)
plt.show()

In [ ]:
# Create time-since-visit animation
# Shows how long since each location was last covered by a balloon
# Green = recently visited, yellow = moderate, red = long ago, gray = never visited
anim = create_time_since_visit_animation(
    fleet,
    analyzer,
    projection='robinson',
    max_hours_colorscale=48,  # Color scale maxes out at 48 hours
    step_size=1,
    save_path=f'time_since_visit_{num_days}_days.mp4',
    fps=24,
    dpi=150,
)

In [ ]:
# Quiver with visible arrows
anim = create_trajectory_animation_parallel(
    fleet,
    wind_field=wind,
    show_wind=True,
    wind_style="quiver",
    arrow_scale=0.5,
    frame_step=1,
    wind_stride=1,
    trail_length=36,
    projection='robinson',
    title_template="Predicted Balloon Positions - Day {day}",
    save_path=f'balloon_trajectories_quiver_{num_days}_days_{len(fleet)}_balloons_robinson_coverage.mp4',  # Save to video file
    fps=24,
    dpi=250,
    show_coverage=True,
    coverage_alpha=0.75,
    n_workers=16,
)

In [ ]:
from balloon_sim.visualization import kepler
kepler._check_kepler()

In [ ]:
from balloon_sim.visualization import export_kepler_html

# Export animated trajectory map
export_kepler_html(
    fleet,
    "trajectories.html",
    animate=True,           # Trip animation layer
    trail_length=0.5,       # Trail length (0–1)
    color_by_balloon=True,  # Different color per balloon
    read_only=True,         # Hide side panel for cleaner view
)


In [ ]:
# Streamlines
anim = create_trajectory_animation(
    fleet,
    wind_field=wind,
    show_wind=True,
    wind_style="streamlines",
    wind_density=2,
    projection='platecarree',
    title_template="Balloon Fleet with Wind - Hour {hour}",
    save_path='balloon_trajectories_streamlines.mp4',  # Save to video file
    fps=24,
    dpi=150
)


In [ ]:
# Create animation WITH wind vectors and save to video
anim_wind = create_trajectory_animation(
    fleet,
    wind_field=wind,
    show_wind=True,
    wind_style='quiver',
    #wind_density=1.5,
    #wind_stride=2,
    wind_scale=1800,
    projection='platecarree',
    trail_length=24,
    interval=50,
    title_template="Balloon Fleet with Wind - Hour {hour}",
    save_path='balloon_trajectories_quiver.mp4',  # Save to video file
    fps=24,
    dpi=150,
)


plt.show()

In [ ]:
dir(wind)

In [ ]:
from balloon_sim.visualization import create_trajectory_animation

# Create a shorter fleet simulation for faster animation
demo_fleet = Fleet.create_random(
    n_balloons=10,
    lat_range=(30, 50),
    lon_range=(-130, -70),
    seed=123
)
demo_fleet.simulate(wind, num_steps=240)  # 10 days

# Create animation without wind vectors
anim = create_trajectory_animation(
    demo_fleet,
    projection='platecarree',
    trail_length=24,
    interval=50,
    title_template="Balloon Fleet - Hour {hour}",
)
plt.show()

## 4b. Animated Trajectory Visualization

Create video animations of balloon trajectories with optional wind vectors.

## 5. Coverage Over Time

Track how coverage builds up over time.

In [ ]:
# Compute coverage at different time steps
time_steps = [24, 48, 168, 336, 720, 1440]  # 1 day, 2 days, 1 week, 2 weeks, 30 days, 60 days
coverage_over_time = []

for step in time_steps:
    grid = analyzer.create_grid()
    for balloon in fleet:
        for i in range(min(step, len(balloon.lats))):
            analyzer.update_coverage(balloon.lats[i], balloon.lons[i], grid, i)
    coverage_pct = analyzer.compute_coverage_percentage(grid) * 100
    coverage_over_time.append(coverage_pct)
    print(f"After {step:4d} hours ({step/24:5.1f} days): {coverage_pct:.1f}% coverage")

## API Summary

```python
from balloon_sim import WindField, Balloon, Fleet, CoverageAnalyzer, download_ncep_data

# Download and load wind data
data_path = download_ncep_data('data/', years=[2023, 2024])
wind = WindField(data_path, interpolation='linear')

# Single balloon
balloon = Balloon(lat=35.0, lon=-100.0, balloon_id='B001')
balloon.simulate(wind, num_steps=720)
df = balloon.to_dataframe()

# Fleet of balloons
fleet = Fleet.create_random(
    n_balloons=30,
    lat_range=(27, 48),
    lon_range=(-120, -81)
)
fleet.simulate(wind, num_steps=1440)

# Coverage analysis
analyzer = CoverageAnalyzer(coverage_radius_km=370)
coverage = fleet.compute_coverage(analyzer)
print(f"Coverage: {analyzer.compute_coverage_percentage(coverage)*100:.1f}%")

# Static visualization
from balloon_sim.visualization import plot_trajectories, plot_coverage
plot_trajectories(fleet)
plot_coverage(coverage)

# Animated visualization with wind vectors
from balloon_sim.visualization import create_trajectory_animation
anim = create_trajectory_animation(
    fleet,
    wind_field=wind,
    show_wind=True,
    save_path='trajectories.mp4',
)
```

## 6. Save Results

Export trajectory data and coverage results.

In [ ]:
# Save fleet trajectories to CSV
fleet_df.to_csv('fleet_trajectories.csv', index=False, float_format='%.4f')
print("Saved fleet trajectories to fleet_trajectories.csv")

# Save coverage grid
np.save('coverage_grid.npy', coverage_grid)
print("Saved coverage grid to coverage_grid.npy")

## API Summary

```python
from balloon_sim import WindField, Balloon, Fleet, CoverageAnalyzer, download_ncep_data

# Download and load wind data
data_path = download_ncep_data('data/', years=[2023, 2024])
wind = WindField(data_path, interpolation='linear')

# Single balloon
balloon = Balloon(lat=35.0, lon=-100.0, balloon_id='B001')
balloon.simulate(wind, num_steps=720)
df = balloon.to_dataframe()

# Fleet of balloons
fleet = Fleet.create_random(
    n_balloons=30,
    lat_range=(27, 48),
    lon_range=(-120, -81)
)
fleet.simulate(wind, num_steps=1440)

# Coverage analysis
analyzer = CoverageAnalyzer(coverage_radius_km=370)
coverage = fleet.compute_coverage(analyzer)
print(f"Coverage: {analyzer.compute_coverage_percentage(coverage)*100:.1f}%")

# Visualization
from balloon_sim.visualization import plot_trajectories, plot_coverage
plot_trajectories(fleet)
plot_coverage(coverage)
```